# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

In [9]:
ETtoday_url = 'https://www.ettoday.net/news/focus/3C%E5%AE%B6%E9%9B%BB/' #3C家電新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [10]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
# scrollWidth和scrollHeight屬性返回元素的整個高度和寬度，包括不可見的高度和寬度（由於溢出）

last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    print('網頁更新中...')
    # 抓到整個瀏覽器高度，然後滑下去
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 等2秒
    time.sleep(2)

    # 計算新高度，與舊的高度比一比，若一樣代表到底了
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        print('到達頁面底端')
        break

    last_height = new_height

網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [20]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [21]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 「社群網站不社群了」...2020年社群媒體五大變革　

▲社群媒體明年將面臨變革。（圖／《ETtoday新聞雲》資料照）
記者陳心怡／綜合外電報導
隨著科技愈來越發達，許多社群媒體爭議也一一浮出，像是隱私權、假新聞等問題，面對即將迎來展新的一年，社群帳號管理平台Hootsuite也發布《2020社群媒體趨勢》年度報告，提出五大該留意的趨勢，不論是單純用戶，還是企業或媒體經營者，都該跟緊未來趨勢，以免陷入過時甚至被淘汰的窘境。
----------------------------------------------------------------------

[1] 【廣編】abc好車網開創中古車交易新紀元　限時入主最高送大金空氣清淨機

▲即日起abc好車網推出顛覆車市活動，限時入主優質中古車享多重限量好禮。
圖、文／大金提供
和泰汽車旗下的abc好車網，順應台灣7成消費者會先上網搜尋中古車相關資訊，才到現場看車的趨勢，即日起只要在abc好車網預付3%購車金，再到該車商店面支付97% 尾款，成交最高送可強效過濾PM2.5的大金空氣清淨機！
本次也特別攜手國內道路救援服務龍頭企業全鋒事業合作，提供成交車輛一年30公里免費道路救援拖吊服務 (若為既有全鋒道路救援者，則改贈全鋒道路救援拖吊服務30公里里程數)，雙品牌聯合出擊保障再升級。
此外，還給愛車完善的保護，成交享車商提供的一年汽車保固服務或指定車款7天不滿意可退車，是直接跟實體車商購買較難享有的額外保障。另外，再加碼送abc 500點（1點等於1元），能於「abc好養車」兌換到車用藍芽發射器、車用充電器等好禮或折抵汽車保養、美容、維修等費用。
----------------------------------------------------------------------

[2] 台手機市場回溫　台灣三星：總體止跌、2020仍須謹慎

▲台灣三星副總經理陳啟蒙。（圖／記者邱倢芯攝）
記者邱倢芯／台北報導
先前有市調機構數據指出，台灣手機市場今年有回溫的跡象；對此，台灣三星副總經理陳啟蒙認為，總體來說今年銷售表現僅能說「止跌」，在面對明年銷售仍

▲Galaxy S系列新機將可能跳號命名。（圖／取自CashKaro）
記者謝仁傑／綜合報導
三星的S系列新機可能不取名為Galaxy S11？爆料達人 冰宇宙（Ice universe）在推特發文，預測三星新機將直接「跳號」到Galaxy S20。
外界猜測，可能是為了不跟iPhone 11撞號，顯得自己落後，另外也有一說，因為明年為2020，剛好象徵S系列從10跨到20的突破性。
蘋果也曾在新手機命名，出現變動，眾所周知，曾在2017、2018跳過數字，改以「X」命名，不過，部分消費者對此卻感到混淆，弄不清楚誰先誰後，今年， 蘋果還是回歸數字命名，推出iPhone 11系列。
三星Galaxy S系列新機，目前預計將於2020年初發表，綜合先前報導，系列機款中螢幕最大的Galaxy S11+（或是取名S12+）將配備6.9吋螢幕，螢幕更新率有機會達120Hz，外界預期該機款將搭載螢幕下指紋感測及Dynamic AMOLED面板。機身尺寸則為166.9mmx76mmx8.8mm，這較Galaxy S10+更長、更寬，也更厚，而更大的機身也為相機升級預留了空間。
----------------------------------------------------------------------

[12] LINE MOBILE推出免費熊大貼圖　申辦499方案送500點再享每月2%點數回饋

▲LINE MOBILE推出歲末年終優惠好康。（圖／LINE MOBILE提供）
▲LINE MOBILE資費方案。（圖／LINE MOBILE提供）
----------------------------------------------------------------------

[13] 宏碁品牌廣告「Swift敢自由篇」　勇奪倫敦國際廣告獎、龍璽創意獎

記者姚惠茹／台北報導
宏碁（Acer）今（25）日宣布網路廣告影片驚艷國際，「Swift敢自由篇」奪得2019英國倫敦國際廣告獎(London International Awards)華文創意競賽數位類獎項及2019 龍璽創意獎(LongXi Creative Awards)品牌娛樂類獎項，是全球唯一獲獎的電腦品牌。
宏碁表示，這次獲獎的網路影片將Swift產品特色「輕薄」轉換成「敢自由」的消費者洞

▲Garmin、Fitbit聖誕節購物提案。（圖／業者提供）
記者邱倢芯／綜合報導
今天就是一年一度的平安夜了，想好要買什麼禮物犒賞辛苦許久的自己了呢？針對今年的聖誕節，Garmin與Fitbit都提出了聖誕提案，要讓你擁有換錶好心情之外，也能獲得健康。
針對較為陽剛的消費者，Garmin推出了全新搭載多樣軍事戰術模式的Instinct本我系列軍事戰術版智慧腕錶；全新軍事戰術版在兩款經典軍事風格的潛行黑、郊狼棕配色外，更推出迷彩灰與迷彩棕兩款為亞洲限定設計款，靈感來自美軍迷彩圖樣，融入迷彩元素與戰地掩蔽色彩概念，全新系列新增軍事戰術模式，跳傘、夜視、潛行模式等軍事作戰功能；兼具日常生活的多種運動模式、智慧通知功能等多樣機能，打造出內在、外型兼具、強悍有型的Instinct本我系列軍事戰術版智慧腕錶，建議售價11,990元。
而針對偏好都會時尚穿搭的消費者，Garmin最新vivomove系列最高階搭載全螢幕隱藏式觸控螢幕與全新設計互動指針，精緻極簡的3針面盤敲敲觸動，喚醒鮮亮的智慧視覺，突破傳統指針僅指引時間刻度，互動指針滑順指向智慧功能顯示，提供全面的健康數據監測與行動支付導入，寬闊的觸控式螢幕允許更完整的訊息提示功能，獨一有二，有美型、有智慧，展現新世代不被定義的態度。
▼針對偏好都會時尚穿搭的消費者，Garmin vivomove系列兼具美型與智慧於一身。（圖／業者提供）
vivomove全系列提供39、42、44毫米多種尺寸與材質變化，共16種不同風格選擇，湛黑、PVD玫瑰金、流沙金、極地銀錶殼，搭配低調奢華風米蘭金屬鍊帶、優雅小牛皮真皮錶帶、極輕編織錶帶或親膚矽膠錶帶，動靜之間隨心所欲掌握。vivomove系列建議售價8,490元至17,990元。
選擇Fitbit裝置作為禮物，代表收禮者也能享有專屬平台與技術，這些功能均經過實證，能幫助使用者發揮最佳身體潛能、保持靈活同時維持健康體重。Fitbit為今年聖誕節準備多款創新產品，不論預算多寡、生活方式需求，都能找到理想的商品。不管是全新頂級Fitbit Versa 2智慧手錶、適合日常生活的Fitbit Inspire HR智慧健身手環，或者是進階版的Fitbit Charge 3智慧健身手環，Fitbit產品能滿足所有人的需求。
此外，在最新的Fitbit OS 4.1版軟體更新中，Fitbit體驗再度升

In [22]:
browser.close()